In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /Users/jwang/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
!pip install torch==1.12.0
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.0 requires torch==1.12.0, but you have torch 2.0.1 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/wizardLM-7B-HF", 
    load_in_8bit=True, 
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("TheBloke/wizardLM-7B-HF")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: Loading binary /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/bitsandbytes/cextension.py:33: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


ImportError: cannot import name 'HF_HUB_DISABLE_SYMLINKS_WARNING' from 'huggingface_hub.constants' (/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/huggingface_hub/constants.py)

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)